In [271]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import random
pd.options.mode.chained_assignment = None  # default='warn'
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
import plotly.express as px
from sklearn import svm
from tkinter import *

In [228]:
data = pd.read_csv('newstats.csv', index_col = 0)

In [229]:
data.groupby(['winner']).mean()[['1 Goals', '2 Goals','1 Possession','2 Possession']]

,1 Goals,2 Goals,1 Possession,2 Possession
winner,,,,
0,0.565891,0.581395,0.474279,0.525721
1,1.407767,0.262136,0.615440,0.384560
2,0.250000,1.090164,0.384262,0.615738


In [242]:
df.groupby(['winner']).mean()[['1 Goals', '2 Goals','1 Possession','2 Possession', "1 Total xG", "2 Total xG"]]

,1 Goals,2 Goals,1 Possession,2 Possession,1 Total xG,2 Total xG
winner,,,,,,
0,0.589147,0.558140,0.486915,0.513085,0.595062,0.622302
1,1.221429,0.257143,0.621036,0.378964,0.956400,0.383886
2,0.256410,1.315018,0.390033,0.609967,0.385414,0.978216


In [226]:
df.groupby(['winner']).count()[['1 Goals', '2 Goals','1 Possession','2 Possession', "1 Total xG", "2 Total xG"]]

,1 Goals,2 Goals,1 Possession,2 Possession,1 Total xG,2 Total xG
winner,,,,,,
0,258,258,258,258,258,258
1,553,553,553,553,553,553
2,553,553,553,553,553,553


In [230]:
preddf = data.copy()

In [239]:
for i in range(5):
    for index, row in preddf.iterrows():
        if bool(random.getrandbits(1)):
            row1 = preddf.iloc[index]
            row1[:21], row1[21:-2] = row1[21:-2], row1[:21].copy()
            row1[-2] = 1 if row1[-2] == 2 else 2 if row1[-2] == 1 else 0
            row1[-1] = 1 if row1[-1] == 2 else 2 if row1[-1] == 1 else 0
            preddf.iloc[index] = row1

In [222]:
preddf2 = preddf.copy()

for index, row in preddf2.iterrows():
    row1 = preddf2.iloc[index]
    row1[:21], row1[21:-2] = row1[21:-2], row1[:21].copy()
    row1[-2] = 1 if row1[-2] == 2 else 2 if row1[-2] == 1 else 0
    row1[-1] = 1 if row1[-1] == 2 else 2 if row1[-1] == 1 else 0
    preddf2.iloc[index] = row1

In [240]:
df = preddf#.append(preddf2)

In [243]:
features = df.iloc[: , :-2]
target = df.winner

In [244]:
X_train, X_test, y_train, y_test = train_test_split(features, target)

In [108]:
cl_estimators = {
    'k-Nearest Neighbor': KNeighborsClassifier(), 
    'Gaussian Naive Bayes': GaussianNB(),
    'Decision Tree': DecisionTreeClassifier(),
    'Logistic Regression': LogisticRegression(max_iter = 100000),
    'SVM': svm.SVC()}

In [109]:
def classifiers_models():
    for name, model in cl_estimators.items(): 
        x = model.fit(X=X_train, y = y_train)
        accuracytest = x.score(X_test, y_test)
        accuracytrain = x.score(X_train, y_train)
        print(name + ": \n\t" + "Classification accuracy on the test data:", f"{accuracytest:.2%}")
        print(f"\tClassification accuracy on the training data: {accuracytrain:.2%}\n")

In [245]:
classifiers_models()

k-Nearest Neighbor: 
	Classification accuracy on the test data: 55.56%
	Classification accuracy on the training data: 73.78%

Gaussian Naive Bayes: 
	Classification accuracy on the test data: 64.33%
	Classification accuracy on the training data: 66.93%

Decision Tree: 
	Classification accuracy on the test data: 59.06%
	Classification accuracy on the training data: 100.00%

Logistic Regression: 
	Classification accuracy on the test data: 69.59%
	Classification accuracy on the training data: 73.97%

SVM: 
	Classification accuracy on the test data: 64.33%
	Classification accuracy on the training data: 68.69%



In [111]:
def logistic_tuning():
    param_grid = {"solver":["sag", "saga","liblinear","newton-cg", "lbfgs"]}
    grid_search = GridSearchCV(LogisticRegression(max_iter = 100000), param_grid, cv=10)
    grid_search.fit(X=X_train, y=y_train)
    print("Best parameters: ", grid_search.best_params_)
    print("Training set score with best parameter: ", grid_search.score(X_train, y_train))
    print("Test set score with best parameter: ", grid_search.score(X_test, y_test))

In [246]:
logistic_tuning()

Best parameters:  {'solver': 'saga'}
Training set score with best parameter:  0.735812133072407
Test set score with best parameter:  0.695906432748538


In [247]:
model = LogisticRegression(max_iter=100000)
model.fit(X=X_train, y = y_train)
predicted = model.predict(X=X_test)
odds = model.predict_proba(X = X_test)


In [248]:
def hwin(s1, s2):
    if s1 > s2: 
        return 1
    elif s2 > s1:
        return 2
    else:
        return 0
X_test['fh winner'] = X_test.apply(lambda row: hwin(row['1 Goals'], row['2 Goals']), axis = 1)

In [249]:
X_test['winner'] = y_test
X_test['predicted'] = predicted

In [250]:
oddf = pd.DataFrame(odds, columns = ['% Tie', '% 1 Win', '% 2 Win'], index = X_test.index)

In [251]:
X_test = pd.concat([X_test, oddf], axis=1)

In [252]:
X_test[X_test['winner'] != X_test['predicted']]

,1 Accurate Long Balls,1 Corners,1 Dribbles Attempted,1 Dribbles Succeeded,1 Fouls Conceded,1 Goals,1 Interceptions,1 Long Balls,1 Offsides,1 Pass Completion,...,2 Total Passes,2 Total Shots on Target,2 Total xG,2 Yellow Cards,fh winner,winner,predicted,% Tie,% 1 Win,% 2 Win
297,26,2,13,6,6,1,4,32,0,0.860,...,319,3,0.863,4,0,1,0,0.767738,0.109271,0.122991
57,12,3,18,10,10,0,2,34,0,0.839,...,136,2,0.594,1,0,2,1,0.245830,0.621076,0.133094
62,18,5,8,5,3,1,3,30,1,0.875,...,123,1,0.139,0,1,0,1,0.093522,0.884987,0.021491
288,8,4,8,3,4,1,5,15,1,0.851,...,228,2,0.891,3,0,1,2,0.225215,0.329735,0.445050
611,22,0,15,13,9,0,5,32,1,0.874,...,240,0,0.072,1,0,0,1,0.273162,0.670670,0.056167
267,24,3,16,9,6,0,6,32,2,0.878,...,148,1,0.306,1,2,2,0,0.365639,0.274987,0.359374
134,18,1,6,5,9,1,9,29,1,0.713,...,404,3,0.420,0,1,0,2,0.124259,0.211019,0.664722
139,14,2,8,6,12,1,7,29,1,0.693,...,364,0,0.338,1,1,1,0,0.461829,0.089320,0.448851
5,9,3,6,6,10,1,2,36,2,0.672,...,309,1,0.156,1,1,0,2,0.121515,0.420842,0.457644
560,21,7,20,17,8,1,3,32,0,0.824,...,124,3,0.562,1,0,0,1,0.042113,0.948407,0.009480


In [189]:
impf = {}
for f, c in zip(features.columns, model.coef_[1]):
    impf[f] = c
impf = dict(sorted(impf.items(), key=lambda x: x[0][1:]) )

In [257]:
coefs = pd.DataFrame(model.coef_, columns = features.columns)

In [367]:
model.predict_proba(X_test.loc[422][:-6].values.reshape(1, -1) )

array([[0.11204974, 0.05501339, 0.83293688]])

In [368]:
X_test.loc[422]

1 Accurate Long Balls       30.000000
1 Corners                    3.000000
1 Dribbles Attempted         9.000000
1 Dribbles Succeeded         7.000000
1 Fouls Conceded             8.000000
1 Goals                      0.000000
1 Interceptions              1.000000
1 Long Balls                35.000000
1 Offsides                   2.000000
1 Pass Completion            0.901000
1 Possession                 0.680000
1 Red Cards                  0.000000
1 Shots                      2.000000
1 Shots Blocked              1.000000
1 Shots woodwork             0.000000
1 Tackles Attempted         13.000000
1 Throws                    13.000000
1 Total Passes             413.000000
1 Total Shots on Target      1.000000
1 Total xG                   0.048000
1 Yellow Cards               1.000000
2 Accurate Long Balls        7.000000
2 Corners                    2.000000
2 Dribbles Attempted        16.000000
2 Dribbles Succeeded         2.000000
2 Fouls Conceded            11.000000
2 Goals     

In [273]:
team1 = ['1 Accurate Long Balls', '1 Corners', '1 Dribbles Attempted',
       '1 Dribbles Succeeded', '1 Fouls Conceded', '1 Goals',
       '1 Interceptions', '1 Long Balls', '1 Offsides', '1 Pass Completion',
       '1 Possession', '1 Red Cards', '1 Shots', '1 Shots Blocked',
       '1 Shots woodwork', '1 Tackles Attempted', '1 Throws', '1 Total Passes',
       '1 Total Shots on Target', '1 Total xG', '1 Yellow Cards']
team2 = ['2 Accurate Long Balls', '2 Corners', '2 Dribbles Attempted',
       '2 Dribbles Succeeded', '2 Fouls Conceded', '2 Goals',
       '2 Interceptions', '2 Long Balls', '2 Offsides', '2 Pass Completion',
       '2 Possession', '2 Red Cards', '2 Shots', '2 Shots Blocked',
       '2 Shots woodwork', '2 Tackles Attempted', '2 Throws', '2 Total Passes',
       '2 Total Shots on Target', '2 Total xG', '2 Yellow Cards']

In [437]:
new = []

In [444]:
width = 800
height = 700
window = Tk()
window.configure(background = 'black')
window.geometry(f'{width}x{height}')

l = Label (window, text = 'Input Game Stats:', bg = 'white', fg = 'black', font = 'none 20 bold')
l.place(x=width/2, y=25)

entries = []

for i in range(len(team1)):
    cv = Entry(window)
    cv.place(x = height/3, y = i * 20 + 100)
    entries.append(cv)
    cvlab = Label(window, text = team1[i], bg = 'white', borderwidth = 1, relief="raised")
    cvlab.place(x = height/10, y = i * 20 + 100)

for i in range(len(team2)):
    cv = Entry(window)
    cv.place(x = height*8/10, y = i * 20 + 100)
    entries.append(cv)
    cvlab = Label(window, text = team2[i])
    cvlab.place(x = height*6/10, y = i * 20 + 100)

odds = []

for i in range(0,3):
    cv = Entry(window)
    cv.place(x = 400, y = i*30 + 550)
    odds.append(cv)
    cvlab = Label(window, text = str(i) + ' win')
    cvlab.place(x = 350, y = i*30 + 550)
    
def stats():
    features = []
    for entry in entries:
        features.append(float(entry.get()))
    new.append(features)
    fm = np.array(features).reshape(1,-1)
    pp = model.predict_proba(fm)
    euro = [float(x.get()) for x in odds]
    print(f'  \tpredicted prob\t\tbook prob \tsize')
    for p, o, win in zip(pp[0], euro, range(0,3)):
        bookp = round(1/o, 3)
        print(f'{win}:\t{round(p,3)}\t\t\t{bookp}\t\t{kelly(p, o) if p > bookp else ""}')
                
button=Button(window,text="Calculate Odds",command=stats)
button.place(x = 100, y = 550)

window.mainloop()

  	predicted prob		book prob 	size
0:	0.168			0.25		
1:	0.01			0.111		
2:	0.822			0.654		0.487


In [413]:
def kelly(p, b):
    return round((p*(b-1) - (1-p))/(b-1), 3)

In [443]:
len(new)

3